# Federated K-Means
In this section, we attempt to form a federated k-means algorithm to preserve the privacy of the individual labs. 

**Federated learning:**  First introduced by Google, federated learning trains an algorithm across multiple decentralized servers/devices with local data samples. There is **no** data sharing between servers. This is in contrast to the standard way of training a machine learning algorithm where all of the training dataset is uploaded to one server. This technique adrress critical issues relating to data privacy and security 

## Diagram of the federated k-means algorithm

In [3]:
from sklearn.cluster import KMeans

In [2]:
class child_kmeans(KMeans):
        def __init__(
            self,
            n_clusters,
            init_centroids='random',
            max_iter=100,
            tol=0.0001,
            distance_metric='euclidean',
            seed=None,
            reassign_min=None,
            reassign_after=None,
            verbose=False,
            batch_size=None,
            sample_fraction=1.0,
            epochs_per_round=1,
            learning_rate=None,
            max_no_change=None,
            adaptive_lr=None,
            momentum=None,
            epoch_lr=1,
    ):
        super().__init__(
            n_clusters=n_clusters,
            init_centroids=init_centroids,
            max_iter=max_iter,
            tol=tol,
            distance_metric=distance_metric,
            seed=seed,
            reassign_min=reassign_min,
            reassign_after=reassign_after,
            verbose=verbose
        )
    
    pass

In [1]:
class parent_kmeans():
    pass

SyntaxError: unexpected EOF while parsing (Temp/ipykernel_36092/3935500515.py, line 1)

## Evaluation 

Before evaluating the results, an important consideration should be noted. As with any K-Means algorithm, results may vary with each run of the algorithm as the algorithm's performance is heavily dependent on the initial clusters chosen. Therefore, the federated-k-means algorithms is run 10 times and  the corresponding evaluation metrics are averaged 10 times. The clusters formed in the previous section as the ground truth. Like-kmeans, we must supply the vector of federated k-me